In [ ]:
import requests
import csv

BASE_NOMINATIM_URL = 'https://nominatim.openstreetmap.org/reverse'
BASE_BIGDATACLOUD_URL = 'https://api.bigdatacloud.net/data/reverse-geocode-client'

input_csv_file = 'geography_without_id.csv' # without id 
output_csv_file = 'geocoded_geography.csv'

def reverse_geocode(latitude, longitude):
    try:
        # Try Nominatim API for reverse geocoding
        nominatim_response = requests.get(BASE_NOMINATIM_URL, params={'lat': latitude, 'lon': longitude, 'format': 'json'})
        nominatim_data = nominatim_response.json()
        address = nominatim_data.get('address', {})
        city = address.get('city', '') or address.get('town', '') or address.get('hamlet', '') or address.get('village', '') or address.get('municipality', '') or address.get('locality', '') or address.get('township', '') or address.get('suburb', '')
        state = address.get('state', '')  
        county = address.get('county', '')

        # If city is not found in Nominatim, use BigDataCloud API
        if not city or state or county:
            bigdatacloud_response = requests.get(BASE_BIGDATACLOUD_URL, params={'latitude': latitude, 'longitude': longitude})
            bigdatacloud_data = bigdatacloud_response.json()
            city = bigdatacloud_data.get('city', '')
            state = bigdatacloud_data.get('principalSubdivision', '')
            county = bigdatacloud_data.get('localAdminArea', '')

        return city, state, county
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

# Open the input CSV file for reading and the output CSV file for writing
with open(input_csv_file, 'r') as csvfile, open(output_csv_file, 'w', newline='') as output_csvfile:
    dataset = csv.DictReader(csvfile)
    fieldnames = dataset.fieldnames + ['city', 'state', 'county']  # Add the new fields to the header

    writer = csv.DictWriter(output_csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in dataset:
        latitude = row['latitude']
        longitude = row['longitude']

        # Call reverse_geocode function to get city, state, and county
        city, state, county = reverse_geocode(latitude, longitude)

        # Update the row with the geocoding results
        row['city'] = city
        row['state'] = state
        row['county'] = county

        # Write the updated row to the output CSV
        writer.writerow(row)
        output_csvfile.flush()



In [ ]:
import reverse_geocoder as rg

def reverse_geo(latitude,longitude): 
# Perform reverse geocoding using reverse_geocoder
    coordinates = ([float(latitude), float(longitude)])
    results = rg.search(coordinates)
    return results[0]

#reverse_geo(37.1502,-86.0439)

### Updating the previous geocoded file (from openstreetnominatim and bigcloud API)

In [ ]:
import csv
import reverse_geocoder as rg



# Input and output file paths
input_csv_file = 'geocoded_FINAL.csv'
output_csv_file = 'geography_reverse_updated_final.csv'

# Open the input CSV file for reading and the output CSV file for writing
with open(input_csv_file, 'r') as geo_final, open(output_csv_file, 'w', newline='') as geo_update:
    reader = csv.DictReader(geo_final)
    fieldnames = ['geo_id'] + reader.fieldnames 
    writer = csv.DictWriter(geo_update, fieldnames=fieldnames)
    writer.writeheader()
    count_id = 0
    id_dict ={}

    for row in reader:
        count_id +=1
        id_dict = {'geo_id': int(count_id)}
        if row['city'] == '' or row['state'] == '' or row['county'] == '':
            latitude = float(row['latitude'])
            longitude = float(row['longitude'])

            # Perform reverse geocoding for missing values
            location = reverse_geo(latitude, longitude)
            if row['city'] == '':
                row['city'] = location['name']
            if row['state'] == '':
                row['state'] = location['admin1']
            if row['county'] == '':
                
                row['county'] = location['admin2']
        row.update(id_dict)
        writer.writerow(row)
        geo_update.flush()

print("Updated missing values in city, state, and county.")


In [ ]:
#Adding ID to the final geocoded file and call it geography_with_id

with open('geography_reverse_update.csv', 'r') as csvfile, open('geography_with_id.csv', 'w', newline='') as new_csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = ['geo_id'] + reader.fieldnames 
    writer = csv.DictWriter(new_csvfile, fieldnames=fieldnames)
    writer.writeheader()
    id_count =0
    id_dict =dict()
    for row in reader:
        id_count +=1
        id_dict={'geo_id':int(id_count)}
        row.update(id_dict)
        writer.writerow(row)


## JUST REVERSE GEOCODER


- just an alternative to the previous solution

In [8]:
import csv
import reverse_geocoder as rg

# Define a function for reverse geocoding
def reverse_geo(coordinates):
    results = rg.search(coordinates)
    return results

# Initialize a list to collect all coordinates
all_coordinates = []

with open('geography.csv', 'r') as geo_final:
    reader = csv.DictReader(geo_final)

    for row in reader:
        latitude = float(row['latitude'])
        longitude = float(row['longitude'])
        coordinates = (latitude, longitude)

            # Append the coordinates to the list
        all_coordinates.append(coordinates)

all_coordinates = tuple(all_coordinates)


In [9]:
import reverse_geocoder
import csv

# Perform reverse geocoding using reverse_geocoder
results = rg.search(all_coordinates)  # Assuming 'all_coordinates' is a list of coordinates

# List to store dictionaries from the CSV file
csv_data = []

# Read data from CSV file
with open('geography.csv', 'r') as geo_final:
    reader = csv.DictReader(geo_final)
    
    # Store CSV data in a list
    for row in reader:
        csv_data.append(row)

# Perform reverse geocoding and combine data
for i, result in enumerate(results):
    # Extract latitude and longitude from CSV data
    latitude = csv_data[i]['latitude']
    longitude = csv_data[i]['longitude']

    # Extract reverse geocoding results
    city = result['name']
    state = result['admin1']
    county = result['admin2']

    # Update CSV data with reverse geocoding results
    csv_data[i].update({'city': city, 'state': state, 'county': county})

# Write combined data to a new CSV file
fieldnames = csv_data[0].keys()  # Assuming the keys are the same for all rows
with open('geography_only_reverse_geocoder.csv', 'w', newline='') as geo_reverse_shit:
    writer = csv.DictWriter(geo_reverse_shit, fieldnames=fieldnames, extrasaction='ignore')
    writer.writeheader()
    writer.writerows(csv_data)

Loading formatted geocoded file...
